In [1]:
import os
import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

/home/asif/anaconda3/envs/medalpaca/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from peft import *

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    # config.base_model_name_or_path,
    "/home/asif/llm/model/medalpaca-13b",
    load_in_8bit=True,
    return_dict=True,
    # quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
# tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained("/home/asif/llm/model/medalpaca-13b")
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 6/6 [00:24<00:00,  4.00s/it]
You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


Memory taken : 14GB 

In [4]:
generation_config = model.generation_config
generation_config.max_new_tokens = 400
generation_config_temperature = 0.3
generation_config.top_p = 0.9
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config_eod_token_id = tokenizer.eos_token_id

In [5]:
DEVICE = "cuda"

In [6]:
def generate_response(ctx: str, options: str) -> str:
    prompt = f"""You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
{ctx}

options:
{options}

reply: 
    """.strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0],skip_special_tokens=True)

    # assistant_start = '<assistant>:'
    # response_start = response.find(assistant_start)
    # return response[response_start + len(assistant_start):].strip()
    return response

In [7]:
print(generate_response("The patient came to the doctor complaining of a persistent headache for the past week.","0: 'The doctor recommended a brain scan to rule out any underlying issues', 1: 'The patient was advised to take painkillers and get plenty of rest', 2: 'The doctor suggested trying acupuncture as an alternative treatment', 3: 'The patient was referred to a neurologist for further examination'"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
The patient came to the doctor complaining of a persistent headache for the past week.

options:
0: 'The doctor recommended a brain scan to rule out any underlying issues', 1: 'The patient was advised to take painkillers and get plenty of rest', 2: 'The doctor suggested trying acupuncture as an alternative treatment', 3: 'The patient was referred to a neurologist for further examination'

reply:
"I'm sorry, but you need to see a neurologist."


Output: 3 ===== Original: 0

In [8]:
print(generate_response("A young child has been experiencing a cough, runny nose, and sneezing for the past two days.","{0: 'The doctor prescribed a cough syrup and advised plenty of fluids', 1: 'The child should avoid exposure to cold temperatures and rest', 2: 'Nasal saline drops can help relieve congestion', 3: 'The doctor recommended a throat swab for a strep test'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A young child has been experiencing a cough, runny nose, and sneezing for the past two days.

options:
{0: 'The doctor prescribed a cough syrup and advised plenty of fluids', 1: 'The child should avoid exposure to cold temperatures and rest', 2: 'Nasal saline drops can help relieve congestion', 3: 'The doctor recommended a throat swab for a strep test'}

reply: {0: 'The doctor prescribed a cough syrup and recommended plenty of fluids.'}

{0: 'The doctor prescribed a cough syrup and recommended plenty of fluids.'}
{0: 'The doctor prescribed a cough syrup and recommended plenty of fluids.'}
{0: 'The doctor prescribed a cough syrup and recommended plenty of fluids.'}
{0: 'The doctor prescribed a cough syrup and recommended plenty of fluids.'}
{0: 'The doctor prescribed a cough syrup and recommended plenty o

Output: 0 (partially) ========= Original: 0

In [9]:
print(generate_response("A woman went to an allergist to determine the cause of her frequent allergic reactions and skin rashes.","{0: 'The allergist suggested conducting an allergy panel test', 1: 'The woman was advised to keep a diary of potential triggers', 2: 'Antihistamine medication can help alleviate symptoms', 3: 'The allergist recommended using hypoallergenic skincare products'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A woman went to an allergist to determine the cause of her frequent allergic reactions and skin rashes.

options:
{0: 'The allergist suggested conducting an allergy panel test', 1: 'The woman was advised to keep a diary of potential triggers', 2: 'Antihistamine medication can help alleviate symptoms', 3: 'The allergist recommended using hypoallergenic skincare products'}

reply: {0: 'The allergist suggested conducting an allergy panel test', 1: 'The allergist recommended hypoallergenic skincare products', 2: 'The allergist recommended hypoallergenic skincare products'}

{0: 'The allergist suggested conducting an allergy panel test', 1: 'The allergist recommended hypoallergenic skincare products'}
{0: 'The allergist suggested conducting an allergy panel test', 1: 'The allergist recommended hypoallergenic 

## XXXXXXXXXX   Does Not follow instruction XXXXXXXXXXXXXXX

# Closed

Output: 1 ====== Original: 1

In [21]:
print(generate_response("The patient visited the clinic for a routine check-up to monitor blood sugar levels and discuss diabetes management.","{0: 'The doctor recommended regular exercise and a balanced diet', 1: 'The patient should monitor blood sugar levels multiple times a day', 2: 'Insulin injections may be necessary to regulate glucose levels', 3: 'The doctor advised scheduling an appointment with a nutritionist'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
The patient visited the clinic for a routine check-up to monitor blood sugar levels and discuss diabetes management.

options:
{0: 'The doctor recommended regular exercise and a balanced diet', 1: 'The patient should monitor blood sugar levels multiple times a day', 2: 'Insulin injections may be necessary to regulate glucose levels', 3: 'The doctor advised scheduling an appointment with a nutritionist'}

reply:0


Output: 0 ====== Original 3

In [22]:
print(generate_response("A man with asthma visited the pulmonologist to adjust his medication and address recent breathing difficulties.","{0: 'The pulmonologist prescribed a stronger bronchodilator for immediate relief', 1: 'The man was advised to use a peak flow meter to monitor lung function', 2: 'Inhaled corticosteroids should be taken daily for long-term control', 3: 'The doctor recommended avoiding triggers such as smoke and pet dander'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A man with asthma visited the pulmonologist to adjust his medication and address recent breathing difficulties.

options:
{0: 'The pulmonologist prescribed a stronger bronchodilator for immediate relief', 1: 'The man was advised to use a peak flow meter to monitor lung function', 2: 'Inhaled corticosteroids should be taken daily for long-term control', 3: 'The doctor recommended avoiding triggers such as smoke and pet dander'}

reply:2


Output: 2 =======  Original 2

In [23]:
print(generate_response("A teenager complained of blurry vision and difficulty reading the board at school.","{0: 'The teenager should undergo an eye exam to check for refractive errors', 1: 'Prescription glasses may be necessary to correct the vision', 2: 'Regular eye exercises can help improve focus and coordination', 3: 'The student was advised to sit closer to the front in the classroom'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A teenager complained of blurry vision and difficulty reading the board at school.

options:
{0: 'The teenager should undergo an eye exam to check for refractive errors', 1: 'Prescription glasses may be necessary to correct the vision', 2: 'Regular eye exercises can help improve focus and coordination', 3: 'The student was advised to sit closer to the front in the classroom'}

reply:1


Output: 1 ===== Original : 1

In [24]:
print(generate_response("A person sought advice from a nutritionist to develop a personalized weight loss plan.","{0: 'The nutritionist recommended a balanced diet with reduced calorie intake', 1: 'Regular physical exercise is essential for weight management', 2: 'Keeping a food journal can help track eating habits', 3: 'The person was advised to set realistic weight loss goals'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A person sought advice from a nutritionist to develop a personalized weight loss plan.

options:
{0: 'The nutritionist recommended a balanced diet with reduced calorie intake', 1: 'Regular physical exercise is essential for weight management', 2: 'Keeping a food journal can help track eating habits', 3: 'The person was advised to set realistic weight loss goals'}

reply:0


Output: 0 ======= Original : 0

In [25]:
print(generate_response("The patient visited the dentist for a routine dental examination and cleaning.","{0: 'The dentist identified a cavity that requires filling', 1: 'Proper brushing and flossing techniques were demonstrated', 2: 'Regular dental check-ups should be scheduled every six months', 3: 'The patient was advised to use a fluoride mouthwash for added protection'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
The patient visited the dentist for a routine dental examination and cleaning.

options:
{0: 'The dentist identified a cavity that requires filling', 1: 'Proper brushing and flossing techniques were demonstrated', 2: 'Regular dental check-ups should be scheduled every six months', 3: 'The patient was advised to use a fluoride mouthwash for added protection'}

reply:2


Output: 2 ======  Original: 2

In [26]:
print(generate_response("A person sought therapy to address anxiety and depression symptoms affecting their daily life.","{0: 'The therapist recommended cognitive-behavioral therapy sessions', 1: 'Medication options were discussed for managing symptoms', 2: 'Practicing mindfulness and relaxation techniques can help reduce anxiety', 3: 'The person was advised to reach out to a support group for additional assistance'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A person sought therapy to address anxiety and depression symptoms affecting their daily life.

options:
{0: 'The therapist recommended cognitive-behavioral therapy sessions', 1: 'Medication options were discussed for managing symptoms', 2: 'Practicing mindfulness and relaxation techniques can help reduce anxiety', 3: 'The person was advised to reach out to a support group for additional assistance'}

reply:1


Output: 1 ========= Original : 1

In [27]:
print(generate_response("A patient visited a dermatologist with concerns about a persistent rash on their face.","{0: 'The dermatologist prescribed a topical corticosteroid for the rash', 1: 'Avoiding harsh skincare products and irritants is recommended', 2: 'Allergy testing may be necessary to identify potential triggers', 3: 'The patient was advised to protect the skin from excessive sun exposure'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A patient visited a dermatologist with concerns about a persistent rash on their face.

options:
{0: 'The dermatologist prescribed a topical corticosteroid for the rash', 1: 'Avoiding harsh skincare products and irritants is recommended', 2: 'Allergy testing may be necessary to identify potential triggers', 3: 'The patient was advised to protect the skin from excessive sun exposure'}

reply:1


Output: 1 ======= Original: 0

In [28]:
print(generate_response("An individual consulted a gastroenterologist regarding recurring stomach pain and digestion problems.","{0: 'The gastroenterologist recommended a comprehensive digestive system evaluation', 1: 'Dietary changes such as reducing spicy foods can alleviate symptoms', 2: 'Medication for acid reflux was prescribed', 3: 'The individual was advised to increase fiber intake for better digestion'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
An individual consulted a gastroenterologist regarding recurring stomach pain and digestion problems.

options:
{0: 'The gastroenterologist recommended a comprehensive digestive system evaluation', 1: 'Dietary changes such as reducing spicy foods can alleviate symptoms', 2: 'Medication for acid reflux was prescribed', 3: 'The individual was advised to increase fiber intake for better digestion'}

reply:1


Output : 1 ======= Original : 2

In [29]:
print(generate_response("A person visited the orthopedic clinic with a suspected fracture in their arm after a fall.","{0: 'An X-ray was performed to confirm the fracture and determine its severity', 1: 'Immobilizing the arm with a cast is necessary for proper healing', 2: 'Pain medication was prescribed to manage discomfort', 3: 'Physical therapy sessions were recommended to regain strength and mobility'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A person visited the orthopedic clinic with a suspected fracture in their arm after a fall.

options:
{0: 'An X-ray was performed to confirm the fracture and determine its severity', 1: 'Immobilizing the arm with a cast is necessary for proper healing', 2: 'Pain medication was prescribed to manage discomfort', 3: 'Physical therapy sessions were recommended to regain strength and mobility'}

reply:1


Output : 1 ========== Original : 1

In [30]:
print(generate_response("A patient sought help from a sleep specialist due to chronic insomnia and difficulty falling asleep.","{0: 'The sleep specialist suggested a sleep study to evaluate sleep patterns', 1: 'Sleep hygiene practices such as a consistent bedtime routine were recommended', 2: 'Prescription medication can be considered for short-term relief', 3: 'The patient was advised to limit caffeine intake and avoid electronics before bed'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A patient sought help from a sleep specialist due to chronic insomnia and difficulty falling asleep.

options:
{0: 'The sleep specialist suggested a sleep study to evaluate sleep patterns', 1: 'Sleep hygiene practices such as a consistent bedtime routine were recommended', 2: 'Prescription medication can be considered for short-term relief', 3: 'The patient was advised to limit caffeine intake and avoid electronics before bed'}

reply:1


Output: 1 ========== Original : 3

In [31]:
print(generate_response("A parent had questions about the recommended vaccination schedule for their newborn baby.", "{0: 'The pediatrician explained the importance of vaccines for the baby's immune system', 1: 'A vaccination chart was provided with the recommended schedule', 2: 'Possible side effects and their rarity were discussed', 3: 'The parent was advised to keep a record of the vaccinations received'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
A parent had questions about the recommended vaccination schedule for their newborn baby.

options:
{0: 'The pediatrician explained the importance of vaccines for the baby's immune system', 1: 'A vaccination chart was provided with the recommended schedule', 2: 'Possible side effects and their rarity were discussed', 3: 'The parent was advised to keep a record of the vaccinations received'}

reply:1


Output : 1 ========== Original : 2

In [32]:
print(generate_response("An individual visited a cardiologist for a check-up to assess heart health and discuss preventive measures.","{0: 'The cardiologist recommended regular cardiovascular exercise', 1: 'A heart-healthy diet low in saturated fats is essential', 2: 'Monitoring blood pressure regularly can help identify potential issues', 3: 'The individual was advised to quit smoking for better heart health'}"))

You are a medical expert who can answer medical related queries. Given the ctx, determine which of the options is correct by doing medical analysis. Answer with only 0,1,2 or 3.

ctx: 
An individual visited a cardiologist for a check-up to assess heart health and discuss preventive measures.

options:
{0: 'The cardiologist recommended regular cardiovascular exercise', 1: 'A heart-healthy diet low in saturated fats is essential', 2: 'Monitoring blood pressure regularly can help identify potential issues', 3: 'The individual was advised to quit smoking for better heart health'}

reply:1


Output : 3 ========= Original : 1

# Result of medalpaca 7b 

Total examples: 15 
Correct Answers: 7
Wrong Answers : 8 